In [1]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
raw_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
first_population_widget = widgets.Text(placeholder='First Pop',description='First Population:', disabled=False)
second_population_widget = widgets.Text(placeholder='Second Pop',description='Second Population:', disabled=False)
display("Upload iQue3 Data (csv)")
display(raw_uploader)
display("Upload ELN w/ Antibodies Sheet (xlsx)")
display(eln_uploader)
display(first_population_widget)
display(second_population_widget)

'Upload iQue3 Data (csv)'

FileUpload(value={}, description='Upload')

'Upload ELN w/ Antibodies Sheet (xlsx)'

FileUpload(value={}, description='Upload')

Text(value='', description='First Population:', placeholder='First Pop')

Text(value='', description='Second Population:', placeholder='Second Pop')

In [2]:
iquename, elnname = list(raw_uploader.value.keys())[0], list(eln_uploader.value.keys())[0]
iquedata = pd.read_csv(io.BytesIO(raw_uploader.value[iquename]['content']))
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
first_population_name = first_population_widget.value
second_population_name = second_population_widget.value
iquedata.rename(columns = {'Plate' : 'iQue3 Plate','Well ID':'iQue3 384 Well ID'}, inplace = True)
ab_sheet.rename(columns = {'Plate' : 'Antibody Plate','Well ID':'Antibody Platemap 96 Well ID'}, inplace = True)
iquedata

,Experiment,Analysis,iQue3 Plate,Export Date and Time,iQue3 384 Well ID,Row,Column,Well Type,Count of All Wells,Median BL1-H of singlets,Median RL1-H of singlets,Count of singlets
0,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 1,9/19/22 12:48,A01,A,1,A2a,5287,5095758.0,482809.0,1145
1,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 1,9/19/22 12:48,B01,B,1,A2a,5520,3333178.0,366352.0,1054
2,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 1,9/19/22 12:48,C01,C,1,A2a,5899,2668647.0,497674.5,886
3,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 1,9/19/22 12:48,D01,D,1,A2a,4855,3441518.5,521828.0,968
4,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 1,9/19/22 12:48,E01,E,1,A2a,4788,2167633.0,413097.0,907
...,...,...,...,...,...,...,...,...,...,...,...,...
1147,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 3,9/19/22 16:13,L24,L,24,Negative,1269,10792.5,5714.0,334
1148,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 3,9/19/22 16:13,M24,M,24,Negative,1235,11082.0,5029.0,249
1149,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 3,9/19/22 16:13,N24,N,24,Negative,1144,11869.5,5698.5,156
1150,22_09_19 Xyphos A2a Primary Screen,Analysis 1,Plate 3,9/19/22 16:13,O24,O,24,Negative,1279,11152.0,4968.0,167


In [3]:
data_cleaned = iquedata[["iQue3 Plate", "iQue3 384 Well ID", "Well Type", "Median RL1-H of singlets", "Count of singlets"]]
data_cleaned

,iQue3 Plate,iQue3 384 Well ID,Well Type,Median RL1-H of singlets,Count of singlets
0,Plate 1,A01,A2a,482809.0,1145
1,Plate 1,B01,A2a,366352.0,1054
2,Plate 1,C01,A2a,497674.5,886
3,Plate 1,D01,A2a,521828.0,968
4,Plate 1,E01,A2a,413097.0,907
...,...,...,...,...,...
1147,Plate 3,L24,Negative,5714.0,334
1148,Plate 3,M24,Negative,5029.0,249
1149,Plate 3,N24,Negative,5698.5,156
1150,Plate 3,O24,Negative,4968.0,167


In [4]:
#Return data frame with only samples of the well type specified
def clean_for_well_type(well_type_string):
    return data_cleaned[data_cleaned["Well Type"] == well_type_string]

#Average function that takes a string indicating the well type of interest (marked in iQue3 software, or manually changed
#Separates n=1 and n=2 replicates from 384 well format, resets indeces, and averages
#Returns a series with the averaged MFI ratios, removing the replicate wells
def average_MFI(well_type_string):
    selected_wells = clean_for_well_type(well_type_string)
    MFIs = selected_wells["Median RL1-H of singlets"]
    firsts = MFIs.iloc[::2]
    seconds = MFIs.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    average = firsts + seconds
    average = average/2
    return average

#Calculates the MFI ratio given a positive and negative series, 
#and appends the MFI ratio to the original data frame, excluding the negative population
#positive_series: the series of the positive values
#negative_series: the series of the negative (untransfected) values
#positive_string: the string that marks the positive samples in the Well Type column from iQue3 csv
def calculate_analysis_sheet_2pop(first_pop_series, second_pop_series, negative_series, first_population_string, second_population_string):
    first_pop_MFI_ratio = first_pop_series/negative_series
    second_pop_MFI_ratio = second_pop_series/negative_series
    original_positive = clean_for_well_type(first_population_string)
    original_positive = original_positive[::2]
    original_positive.reset_index(inplace = True, drop = True)
    original_positive[first_population_string + " MFI Ratio"] = first_pop_MFI_ratio
    original_positive[first_population_string + " Averaged Raw MFI"] = first_pop_series
    original_positive[second_population_string + " MFI Ratio"] = second_pop_MFI_ratio
    original_positive[second_population_string + " Averaged Raw MFI"] = second_pop_series
    original_positive["Parental Averaged Raw MFI"] = negative_series
    analysis_sheet = pd.concat([original_positive, ab_sheet], axis = 1)
    analysis_sheet.index += 1
    analysis_sheet.drop(columns = ['Median RL1-H of singlets'], inplace = True)
    return analysis_sheet
# analysis_sheet[["iQue3 Plate", "iQue3 384 Well ID", "Well Type", first_population_string + " MFI Ratio", 
#                            second_population_string + " MFI Ratio", 
#                            "Antibody", "Concentration"]] 

In [5]:
averageFirstPopulation = average_MFI(first_population_name)
averageSecondPopulation = average_MFI(second_population_name)
averageParental = average_MFI("Negative")

In [6]:
#Create new DF for MFI ratios(don't modify originals)
analyzed_data = calculate_analysis_sheet_2pop(averageFirstPopulation, averageSecondPopulation,
                                        averageParental, first_population_name, second_population_name)
analyzed_data.to_excel("Flow Binding Primary Screen.xlsx", sheet_name='Analyzed Data')
analyzed_data
# AllLeadsSorted = analyzed_data.sort_values(by= first_population_name + " MFI Ratio", ascending = False)
# AllLeadsSorted

,iQue3 Plate,iQue3 384 Well ID,Well Type,Count of singlets,A2a MFI Ratio,A2a Averaged Raw MFI,A2b MFI Ratio,A2b Averaged Raw MFI,Parental Averaged Raw MFI,Antibody Plate,Lot #,Protein Name,Antibody Platemap 96 Well ID,Protein Concentration,BPP
1,Plate 1,A01,A2a,1145,9.465834,424580.50,0.376984,16909.25,44854.00,BPP577 IGGP337 Plate 1,55181,TB206-1_hIgG4,A1,0.03,577
2,Plate 1,C01,A2a,886,51.013385,509751.25,1.231173,12302.50,9992.50,BPP577 IGGP337 Plate 1,55182,TB762-001,B1,0.39,577
3,Plate 1,E01,A2a,907,18.431714,400558.00,0.932979,20275.50,21732.00,BPP577 IGGP337 Plate 1,55183,TB762-002,C1,0.15,577
4,Plate 1,G01,A2a,1075,0.562598,222485.75,0.540220,213636.25,395461.25,BPP577 IGGP337 Plate 1,55184,TB762-003,D1,0.08,577
5,Plate 1,I01,A2a,858,0.426394,702561.25,0.535821,882863.00,1647682.50,BPP577 IGGP337 Plate 1,55185,TB762-004,E1,0.04,577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Plate 3,G08,A2a,761,2.472081,115154.50,3.031976,141235.50,46582.00,BPP603,55149,PRJ134-H093,D12,0.03,603
189,Plate 3,I08,A2a,1139,1.870828,161201.75,2.287477,197102.75,86166.00,BPP603,55150,PRJ134-095,E12,0.02,603
190,Plate 3,K08,A2a,721,1.128313,6716.00,1.051451,6258.50,5952.25,BPP603,54985,TB194-11_IgG4 added in,NaN,0.65,577
191,Plate 3,M08,A2a,706,1.200513,6439.25,1.088604,5839.00,5363.75,BPP603,NaN,Traz,G12,0.42,57722_09_19 Xyphos A2a Primary Screen\tAnalysi...


In [ ]:
# #Sort DF by highest MFI ratio (top hits)
# AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
# Top10 = MFIRatio_Antibody_cleaned.nlargest(10, "MFI Ratio")
# #Top10 = Top10.append(MFIRatio_Antibody_cleaned.loc[MFIRatio_Antibody_cleaned["DNA"] == "SC-175-017"])
# fig, ax = plt.subplots()
# ax.scatter(x = AllLeadsSorted["Protein Concentration (ug/uL)"], y=AllLeadsSorted["MFI Ratio"], alpha = .2, color = "blue")
# ax.scatter(x = Top10["Protein Concentration (ug/uL)"], y = Top10["MFI Ratio"], alpha = .8, color = "blue", label = "Top Hits")
# ax.set_title("CD40 Primary Screen")
# ax.set_xlabel("Concentration (mg/ml)")
# ax.set_ylabel("MFI Ratio \n Transfected/Untransfected")
# ax.legend()

# texts = []
# for row in Top10.itertuples(index = False):
#     ab, conc, MFI = row[4], row[5], row[3]
#     ax.annotate(row[4], xy = (conc, MFI), textcoords = "offset points", xytext = (8, -2), ha ='left')
#     #texts.append(plt.text(ab, conc, MFI))